In [1]:
from source.tinydb_manager import TinyDB_manager
from benchmark.LLM_benchmark import AnswerEvaluator
import pandas as pd
import os

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package punkt to C:\Users\Igorexy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Igorexy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# input_file = "llama_3.2_3b_instruct.json"
input_file = "mistral_7b_instruct_v0.3.json"
# input_file = "qwen2.5_7b_instruct.json"
# input_file = "qwen2.5_7b_instruct_1m.json"
# input_file = "qwen3_1.7b.json"

input_file = input_file.replace('-', '_')

In [6]:
tiny_config = f"benchmark/output_LLM/{input_file}"

tiny_manager = TinyDB_manager(tiny_config)
bench = AnswerEvaluator(data=tiny_manager.export_json())
metrics = bench.evaluate()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
merged_dict = {**metrics[0], **metrics[2]}
df = pd.DataFrame.from_dict(merged_dict, orient='index', columns=['score'])

# Путь и имя выходного файла
output_dir = 'benchmark/output_LLM'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, input_file.replace('.json', '.xlsx'))

# Сохранение в Excel
df.to_excel(output_file)

# Чтение всех Excel-файлов из папки и объединение

In [9]:
# --- Чтение всех Excel-файлов из папки и объединение ---

output_dir = 'benchmark/output_LLM'
# Получаем список всех .xlsx файлов
files = [f for f in os.listdir(output_dir) if f.endswith('.xlsx')]

# Загружаем каждый файл в DataFrame и переименовываем колонку в имя файла без расширения
dfs = []
for file in files:
    filepath = os.path.join(output_dir, file)
    df = pd.read_excel(filepath, index_col=0)
    df.columns = [os.path.splitext(file)[0]]  # переименование колонки
    dfs.append(df)

# Объединение всех по индексу (метрикам)
final_df = pd.concat(dfs, axis=1)


final_df.to_excel("benchmark/comparison_llm.xlsx")
# Пример вывода
final_df

,mistral_7b_instruct_v0.3,qwen2.5_7b_instruct,qwen2.5_7b_instruct_1m,qwen3_1.7b
f1_avg,0.084057,0.273534,0.248034,0.206777
rouge_l_avg,0.096896,0.311074,0.268135,0.248296
rouge_n_avg,0.241753,NaN,NaN,0.529027
bleu_avg,0.007732,0.078013,0.060993,0.050352
cosine_similarity_avg,0.229010,0.761226,0.715111,0.636403
bertscore_f1_avg,-0.070083,0.205342,0.135657,0.132355
quality_score_avg,0.101447,0.359055,0.313357,0.313448
gen_time_avg,9.571894,302.239571,300.215431,15.059553
gen_time_median,9.032991,280.448153,296.048041,11.818631
gen_time_95th_percentile,11.553661,724.604866,475.442678,17.148585
